In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [7]:
from typing import List
from pydantic import BaseModel, Field
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents.react.agent import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain.tools import tool
from langchain_classic import hub
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


In [4]:
class Source(BaseModel):
    """Schema for the source of the agent"""
    url : str = Field(description="The url of the source")

In [5]:
class AgentResponse(BaseModel):
    """Schema for agent respons with answer and sources."""
    answer : str = Field(description="The agent's answer to the query")
    sources : List[Source] = Field(default_factory=list, description="The sources used by the agent to answer the query.")

In [29]:
REACT_PROMPT_WITH_FORMAT_INSTRUCTIONS = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question formatted according to the format instructions: {format_instructions}

Begin!

Question: {input}
Thought:{agent_scratchpad}

IMPORTANT:
You must respond with exactly ONE of the following:
- An Action and Action Input
- OR a Final Answer
Never both in the same response.

"""

In [30]:
tool = [TavilySearch()]
llm = ChatOpenAI(model="openai/gpt-4o-mini", temperature=0)
output_parser = PydanticOutputParser(pydantic_object=AgentResponse)
react_prompt_with_format_instructions = PromptTemplate(
    template=REACT_PROMPT_WITH_FORMAT_INSTRUCTIONS,
    input_variables=["input", "agent_scratchpad", "tools", "tool_names", "format_instructions"],
).partial(format_instructions = output_parser.get_format_instructions())

In [33]:
agent = create_react_agent(llm, tool, prompt=react_prompt_with_format_instructions)
agent_executor = AgentExecutor(agent=agent, tools=tool, handle_parsing_errors=True, verbose=True)
chain = agent_executor
result = chain.invoke({"input": "Search for 3 ai jobs in india"})



> Entering new AgentExecutor chain...
Action: tavily_search  
Action Input: "AI jobs in India"  {'query': 'AI jobs in India', 'response_time': 1.03, 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://cutshort.io/jobs/artificial-intelligence-ai-jobs', 'title': '50+ Artificial Intelligence (AI) Jobs in India - Cutshort', 'content': "We are seeking Full Stack Developers with strong experience in building and enhancing CRM-based applications, combined with hands-on exposure to AI-driven tools and modern development practices. We are looking for a rockstar Inside Sales Executive to work with our Sales and Marketing team to develop and execute a healthy lead generation program, driving qualified leads into the sales pipeline. We are looking for a Senior AI / ML Engineer to join our fast-growing team and help build AI-driven data platforms and intelligent solutions. We are seeking a Senior Test Automation Engineer (SDET) with expertise in\xa0AI/LLM testi

In [45]:
print(result["input"])
print(result["output"])
print(type(result))
print((result.keys()))


Search for 3 ai jobs in india
{"answer":"Here are three AI jobs currently available in India:\n\n1. **Senior AI / ML Engineer** - Join a fast-growing team to help build AI-driven data platforms and intelligent solutions. Requires experience in AI/ML and strong programming skills. [More details here](https://cutshort.io/jobs/artificial-intelligence-ai-jobs).\n\n2. **Data Scientist** - A role focused on data analysis and machine learning, available through various companies. [Explore more jobs here](https://in.linkedin.com/jobs/artificial-intelligence-jobs).\n\n3. **AI Engineer** - Opportunities for AI Engineers are available, with a focus on remote positions. [Check out remote jobs here](https://www.crossover.com/jobs/ai-engineer/in).","sources":[{"url":"https://cutshort.io/jobs/artificial-intelligence-ai-jobs"},{"url":"https://in.linkedin.com/jobs/artificial-intelligence-jobs"},{"url":"https://www.crossover.com/jobs/ai-engineer/in"}]}
<class 'dict'>
dict_keys(['input', 'output'])


In [46]:
output_parser.parse(result["output"])

AgentResponse(answer='Here are three AI jobs currently available in India:\n\n1. **Senior AI / ML Engineer** - Join a fast-growing team to help build AI-driven data platforms and intelligent solutions. Requires experience in AI/ML and strong programming skills. [More details here](https://cutshort.io/jobs/artificial-intelligence-ai-jobs).\n\n2. **Data Scientist** - A role focused on data analysis and machine learning, available through various companies. [Explore more jobs here](https://in.linkedin.com/jobs/artificial-intelligence-jobs).\n\n3. **AI Engineer** - Opportunities for AI Engineers are available, with a focus on remote positions. [Check out remote jobs here](https://www.crossover.com/jobs/ai-engineer/in).', sources=[Source(url='https://cutshort.io/jobs/artificial-intelligence-ai-jobs'), Source(url='https://in.linkedin.com/jobs/artificial-intelligence-jobs'), Source(url='https://www.crossover.com/jobs/ai-engineer/in')])

In [50]:
sour=output_parser.parse(result["output"]).sources
ans=output_parser.parse(result["output"]).answer

In [51]:
from pprint import pprint
pprint(ans)
pprint(sour)

('Here are three AI jobs currently available in India:\n'
 '\n'
 '1. **Senior AI / ML Engineer** - Join a fast-growing team to help build '
 'AI-driven data platforms and intelligent solutions. Requires experience in '
 'AI/ML and strong programming skills. [More details '
 'here](https://cutshort.io/jobs/artificial-intelligence-ai-jobs).\n'
 '\n'
 '2. **Data Scientist** - A role focused on data analysis and machine '
 'learning, available through various companies. [Explore more jobs '
 'here](https://in.linkedin.com/jobs/artificial-intelligence-jobs).\n'
 '\n'
 '3. **AI Engineer** - Opportunities for AI Engineers are available, with a '
 'focus on remote positions. [Check out remote jobs '
 'here](https://www.crossover.com/jobs/ai-engineer/in).')
[Source(url='https://cutshort.io/jobs/artificial-intelligence-ai-jobs'),
 Source(url='https://in.linkedin.com/jobs/artificial-intelligence-jobs'),
 Source(url='https://www.crossover.com/jobs/ai-engineer/in')]
